In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [8]:
dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [9]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [10]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]

In [11]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [12]:
dependent=dataset[['Purchased']]

In [13]:
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [14]:
independent.shape

(400, 3)

In [15]:
dependent.shape

(400, 1)

In [16]:
from sklearn.model_selection import  train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [17]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
from sklearn.model_selection import GridSearchCV
param_grid = {'solver':['newton-cg','lbfgs', 'liblinear', 'saga'],'penalty':['l2']}
grid=GridSearchCV(LogisticRegression(),param_grid,refit = True, verbose = 3, n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

C:\Users\ramsm\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done   6 out of  12 | elapsed:    2.5s remaining:    2.5s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    2.6s finished
C:\Users\ramsm\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_weighted', verbose=3)

In [20]:
re = grid.cv_results_
grid_prediction = grid.predict(X_test)

In [21]:
from sklearn.metrics import confusion_matrix
cf=confusion_matrix(y_test,grid_prediction)

In [22]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)

In [ ]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print("the f1_macro value for the best parameter {}:".format(grid.best_params_),f1_macro)

In [23]:
print ("the confusion Matrix:\n",cf)

the confusion Matrix:
 [[80  5]
 [12 37]]


In [24]:
print ("the report:\n",clf_report)

the report:
               precision    recall  f1-score   support

           0       0.87      0.94      0.90        85
           1       0.88      0.76      0.81        49

    accuracy                           0.87       134
   macro avg       0.88      0.85      0.86       134
weighted avg       0.87      0.87      0.87       134



In [25]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9498199279711884

In [26]:
table=pd.DataFrame.from_dict(re)

In [27]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.012305,0.003509,0.002016,0.000004,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.781933,0.729564,0.905338,0.805434,0.073438,2
1,0.008335,0.005371,0.004224,0.003535,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.781933,0.729564,0.905338,0.805434,0.073438,2
2,0.007698,0.002791,0.002763,0.001976,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.794573,0.729564,0.917693,0.813798,0.077739,1
3,0.006226,0.002282,0.001873,0.001337,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.781933,0.729564,0.905338,0.805434,0.073438,2
